In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [43]:
class SNNA(nn.Module):
    def __init__(self) -> None:
        super(SNNA, self).__init__()
        self.sl1 = nn.Linear(26,12)
        self.sl2 = nn.Linear(12,9)

    def forward(self, x):
        x = F.relu(self.sl1(x))
        return F.relu(self.sl2(x))

In [44]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [45]:
import pandas as pd
data = pd.read_csv('/drive/My Drive/Data/word_list.csv')
data.head()

,words,love,disgust,courage,fear,anger,laughter,surprise,sorrow,peace
0,acceptance,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,admiration,0.8,0.0,0.3,0.0,0.0,0.2,0.1,0.0,0.8
2,adoration,0.9,0.0,0.1,0.0,0.0,0.3,0.1,0.0,0.9
3,affection,1.0,0.0,0.0,0.0,0.0,0.4,0.1,0.0,1.0
4,afraid,0.0,0.3,0.3,1.0,0.3,0.0,0.0,0.3,0.0


In [71]:
words = data['words'].tolist()
words_repr = [prepare_input(word) for word in words]

In [72]:
parameters = []
for i in range(len(words)):
  row = data.iloc[i].tolist()
  row = row[1:]
  row = [float(n) for n in row]
  parameters.append(row)

In [84]:
# batch_size = 50
training_data = [
                 (words_repr[:50], parameters[:50]),
                 (words_repr[50:100], parameters[50:100]),
                 (words_repr[100:150], parameters[100:150]),
                 (words_repr[150:200], parameters[150:200]),
]
testing_data = (words_repr[200:267], parameters[200:267])

In [74]:
def prepare_input(word):
  alphabets = 'abcdefghijklmnopqrstvuwxyz'
  vector = [float(0) for x in range(26)]
  for c in word:
    try:
      vector[alphabets.index(c)] += 1.00
    except:
      continue
  return vector

In [75]:
model = SNNA()
learning_rate = 1e-3
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(M.parameters(), lr=learning_rate)

In [80]:
def train_loop():
  count = 1
  for batch in training_data:
    X = torch.tensor(batch[0])
    Y = torch.tensor(batch[1])

    pred = model(X)
    loss = loss_fn(pred, Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Batch: {count} Loss: {loss.item()}")
    count += 1

In [109]:
def test_loop():
  X = torch.tensor(testing_data[0])
  Y = torch.tensor(testing_data[1])
  test_loss = 0

  with torch.no_grad():
    pred = model(X)
    test_loss = loss_fn(pred, Y).item()
  
  print(f"Average Loss: {test_loss}")

In [103]:
train_loop()

Batch: 1 Loss: 0.10056008398532867
Batch: 2 Loss: 0.12324883043766022
Batch: 3 Loss: 0.09638093411922455
Batch: 4 Loss: 0.07687357813119888


In [110]:
test_loop()

Average Loss: 0.059614263474941254


In [113]:
torch.save(model.state_dict(), 'model_weights.pth')